# 第3回：画像分類とGrad-CAMによるAI判断の可視化

## 学習目標
- Grad-CAMを用いて、AIが判断時に画像のどの部分に注目しているかを可視化する

## Grad-CAMとは？
Grad-CAM（Gradient-weighted Class Activation Mapping; 勾配重み付けクラス活性化マッピング）は、
「深層学習モデルが判断を下す際に画像のどの部分に注目したか」を可視化する技術です。
- 赤色領域 → AIが強く注目した領域
- 青色領域 → AIが比較的注目しなかった領域


---
## はじめに

**操作方法**: `Shift + Enter` でセルを順番に実行してください。セッションが切断された場合は、上から再実行すれば復旧できます。

---

### この講義で挑戦すること

AIが「この画像は異常です」と判定したとき、あなたは信じられますか？

医療現場では「なぜそう判断したのか」を説明できないAIは信頼されません。ブラックボックスのままでは、医師も患者も納得できないからです。

この講義では、**Grad-CAM**という技術を使って、AIが画像の「どこを見て」判断しているのかを可視化します。AIの頭の中を覗いてみましょう。

---
## Step 1: 環境構築

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")


---
## 二値分類とは

二値分類は、データを2つのカテゴリに分ける最もシンプルな分類タスクです。本演習では「正常」か「異常」かを判定します。

メールの「迷惑メールかどうか」、商品レビューの「ポジティブかネガティブか」なども二値分類です。Yes/Noで答えられる質問に対応しており、多クラス分類と比べて結果の解釈が直感的です。

---
## Step 2: データ準備とモデルトレーニング

ChestMNISTデータセット（胸部X線画像）を使用します。

元のデータセットには14種類の疾患ラベルがありますが、本演習では**二値分類**に変換します
- **正常 (0)**: 異常なし
- **異常 (1)**: 何らかの異常が存在する

In [ ]:
# 胸部画像データのロード（ChestMNIST）
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess(
    'chestmnist', 
    binary_classification=True
)

print(f"✓ 訓練データ: {x_train.shape}")
print(f"ラベル一覧: {info['label']}")

In [ ]:
# 二値分類モデルの構築と学習
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=1  # Binary classification: 1 output unit with sigmoid
)

history = model.fit(x_train, y_train, epochs=5, validation_split=0.1, batch_size=128)

---
## Step 3:  Grad-CAMによる可視化

訓練済みモデルが診断のために画像のどの部分に焦点を当てているか見てみましょう

In [ ]:
# 最初の症例へのGrad-CAMの可視化
exercise_logic.show_gradcam(
    model, 
    x_test[0], 
    title_original="Chest X-ray",
    title_gradcam="AI Focus Region"
)

In [ ]:
# 複数の画像を比較する（最初の8症例）
exercise_logic.show_gradcam_comparison(
    model, 
    x_test[:8], 
    cols=4
)

---
### 練習：さまざまな画像を試す
`x_test[0]`の`0`を変更すると、異なる画像に対するGrad-CAMを表示できます。

In [ ]:
# 例：10症例目の画像を試してみてください
exercise_logic.show_gradcam(model, x_test[10])

### 練習：以下の点を検討してください

1. AIは胸部X線写真のどの領域に焦点を当てていますか？
2. これらの焦点領域は医学的に妥当であると考えますか？
3. Grad-CAMのような説明可能なAI技術は、臨床現場でどのように有用であると考えられますか？

---
## 概要

- **二値分類**：画像を二つのカテゴリ（正常 vs 異常）に分類する手法
- **Grad-CAM**：CNNが「どこを見ているか」を可視化する技術
- **説明可能なAI（XAI）**：医療AIへの信頼構築に不可欠な技術

---
## 考察課題の回答例

以下は考察課題に対する回答の一例です。これが唯一の正解ではなく、議論の出発点として活用してください。

### 1. AIが焦点を当てる領域（胸部X線）

胸部X線では、可視化上は肺野・心陰影・横隔膜などの領域が注目されることが多い一方で、モデルが背景や画像周辺の情報に反応する場合もあり得る：

- 病変がある場合、理想的には病変周囲が強調されることが望ましい
- ただし、撮影条件やマーカー、周辺情報などに依存した"ショートカット特徴"を学習する可能性がある

### 2. 焦点領域の医学的妥当性の評価

注目領域の妥当性は慎重に評価する：

- 解剖学的に意味のある領域（肺、心臓など）に注目しているかを確認する
- 画像の端・背景・文字情報などが強く出る場合は、不適切な特徴学習の可能性を疑う
- 医師の注目領域との比較や、施設・機器を跨いだ外部検証で一貫性を確かめる

### 3. 説明可能なAI（例：Grad-CAM）の位置づけ

Grad-CAM等は有用だが、限界を理解したうえで使うべき：

- **デバッグ/品質管理**: 不適切な領域への依存を発見する助けになる
- **教育的利用**: 所見と対応する領域の議論材料になり得る
- **注意点**: 可視化は粗く、必ずしも「因果的根拠」を保証しないため、性能評価や外部検証とセットで解釈する
- **規制・品質の観点**: 説明可能性は万能要件ではなく、意図された使用とリスクに応じて、透明性・評価・監視などを総合的に整える

---
## 発展的な学習（技術的詳細に興味のある方へ）

この講義では、技術的な詳細を `exercise_logic.py` に分離しています。
より深く学びたい方は、以下の関数のソースコードを参照してください。

### この講義で使用した主要関数

| 関数名 | 機能 | 技術的なポイント |
|--------|------|------------------|
| `initialize_environment()` | 環境セットアップ | Colab/Local判定、GPU設定 |
| `load_and_preprocess(binary_classification=True)` | 二値分類用データ変換 | マルチラベル→二値変換の仕組み |
| `build_model()` | CNNモデル構築 | last_conv_layerの命名とGrad-CAMとの連携 |
| `compute_gradcam()` | Grad-CAM計算 | 勾配計算、重み付き和、ReLU適用 |
| `show_gradcam()` | Grad-CAM可視化 | ヒートマップのオーバーレイ |
| `show_gradcam_comparison()` | 複数画像の比較 | バッチ処理による効率化 |

### ソースコードの参照方法

`exercise_logic.py` はGitHubリポジトリで公開しています：

https://github.com/kshimoji8/MedMNIST-Exercise/blob/main/exercise_logic.py

各関数には詳細な技術解説をdocstring（関数冒頭のコメント）として記載しています。